In [3]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [4]:
df_final = pd.read_csv('data_clean/df_final_before_preprocessing.csv')

- Le DataFrame contient 307 507 lignes qui sont nos clients, et 341 colonnes (dont l'ID du client et sa target)

In [8]:
df_final.shape

(307507, 341)

In [9]:
df_final.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATI

# Preparation before modelling

## Separation train and test data

In [10]:
df_final.TARGET.value_counts()

0    282682
1     24825
Name: TARGET, dtype: int64

In [11]:
df_final.TARGET.value_counts()/df_final.shape[0]

0    0.91927
1    0.08073
Name: TARGET, dtype: float64

In [12]:
target = df_final.TARGET
inputs = df_final.drop(columns=['TARGET','SK_ID_CURR'])
inputs_with_ids = df_final.drop(columns=['TARGET'])

- Séparation train(70 %) et test(30 %) des données

In [13]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.3, random_state=1)

In [14]:
X_train.shape, X_test.shape

((215254, 339), (92253, 339))

- On sauvegarde en csv les données test qui seront nos clients d'exemples pour API et dashboard

In [15]:
index_test = X_test.index.tolist()
X_test_with_id = inputs_with_ids.loc[index_test]

In [21]:
X_test_with_id.to_csv('Customer_data_test.csv',index=False)

## Preprocessing

On sépare:
- les colonnes avec des entiers - variables continues dont des catégorielles (devenues continues par encodage durant le feature engineering)
- des colonnes avec des float - variables discrètes

In [18]:
columns_int = list(X_train.select_dtypes(include=['uint8', 'int64']).columns)
columns_float = list(X_train.select_dtypes(include=['float64']).columns)

In [19]:
X_train_preprocess = X_train.copy()
X_test_preprocess = X_test.copy()

Application de SimpleImputer pour remplacer les Nan selon 2 stratégies:
- Si la variable est continue, on remplace Nan par la valeur la plus fréquente pour cette variable.
- Si la variable est discrète, on remplace Nan par la moyenne des valeurs pour cette variable.

Bien sûr, on Fit le modèle par les données train, et on transforme ensuite les données train et test.

In [20]:
from sklearn.impute import SimpleImputer

imp_most_frequent = SimpleImputer(strategy="most_frequent")
imp_mean = SimpleImputer(strategy="mean")
imp_most_frequent.fit(X_train_preprocess[columns_int])
imp_mean.fit(X_train_preprocess[columns_float])

X_train_preprocess[columns_int] = imp_most_frequent.transform(X_train_preprocess[columns_int])
X_train_preprocess[columns_float] = imp_mean.transform(X_train_preprocess[columns_float])
X_test_preprocess[columns_int] = imp_most_frequent.transform(X_test_preprocess[columns_int])
X_test_preprocess[columns_float] = imp_mean.transform(X_test_preprocess[columns_float])

- Application de RobustScaler pour scaler les données contenant de nombreux outliers. Fit par les données train et transformation des données train et test.

In [21]:
from sklearn.preprocessing import RobustScaler

rs_scaler = RobustScaler()
rs_scaler.fit(X_train_preprocess[columns_float])

X_train_preprocess[columns_float] = rs_scaler.transform(X_train_preprocess[columns_float])
X_test_preprocess[columns_float] = rs_scaler.transform(X_test_preprocess[columns_float])

In [22]:
X_train_preprocess.shape, X_test_preprocess.shape

((215254, 339), (92253, 339))

## Pipeline preprocessing

- Création d'une fonction de preprocessing réunissant l'imputer et le scaler précédant dans le but de sauvegarder ce modèle de preprocessing créé (sauvegarde fin du notebook).
Comme d'habitude, le processor est fité sur les données train, et il transforme ensuite les données train et test.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Pipeline data transformation (Imputation / Scaling):
def Preprocessing (numeric,categoric):
    numeric_trans = [('imputer',SimpleImputer(strategy= 'mean')),('scaler', RobustScaler())]
    numeric_pipeline = Pipeline(numeric_trans)
    categoric_trans = [('imputer',SimpleImputer(strategy="most_frequent"))]
    categoric_pipeline = Pipeline(categoric_trans)
    all_trans = [("numeric",numeric_pipeline,numeric),("categorical",categoric_pipeline,categoric)]
    preprocessor = ColumnTransformer(all_trans, remainder='passthrough')
    return preprocessor

In [28]:
# Data Transformed
preprocessor_fitted = Preprocessing(columns_float,columns_int).fit(X_train)

In [29]:
X_train_preprocess = preprocessor_fitted.transform(X_train)
X_test_preprocess = preprocessor_fitted.transform(X_test)

In [30]:
X_train_preprocess.shape, X_test_preprocess.shape

((215254, 339), (92253, 339))

## Resampling

Les données sont largement déséquilibrée:
- environ 92 % des données sont des clients à qui on accorde le prêt (classe 0 majoritaire)
- environ 8 % des données sont des clients à qui on refuse le prêt

In [26]:
y_train.value_counts(), y_train.value_counts()/y_train.shape

(0    197857
 1     17397
 Name: TARGET, dtype: int64,
 0    0.919179
 1    0.080821
 Name: TARGET, dtype: float64)

#### from imblearn.under_sampling import RandomUnderSampler
- Le sous-échantillonnage aléatoire (random undersampling) des observations majoritaires: on retire aléatoirement des observations majoritaires

#### from imblearn.under_sampling import TomekLinks
- Le sous-échantillonnage synthétique (TomekLinks) des observations majoritaires: on retire des observations majoritaires, ressemblantes à des minoritaires (des majoritaires ambigues)

#### from imblearn.over_sampling import RandomOverSampler
- Le sur-échantillonnage aléatoire (random oversampling) des observations minoritaires : on tire au hasard des individus minoritaires (on les dupplique) que l’on rajoute aux données.

#### from imblearn.over_sampling import SMOTE
- Le sur-échantillonnage synthétique (SMOTE pour Synthetic Minority Oversampling Technique) produit des observations minoritaires ressemblantes mais distinctes de celles déjà existantes.

- Nous allons combiner SMOTE avec RandomUnderSampler pour rééquilibré les classes.
J'utilise pas Tomek car cela prend beaucoup trop de temps par rapport à la valeur ajoutée.
On applique cette méthoque que sur les données train.

In [30]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
#from imblearn.under_sampling import TomekLinks
#from imblearn.combine import SMOTETomek

In [31]:
## Over-sampling
smote = SMOTE(sampling_strategy=0.4)

## Under-sampling 
#tomek = TomekLinks(sampling_strategy='majority')
rus = RandomUnderSampler(sampling_strategy=0.6)

## Over-sampling then under-sampling
#smtomek = SMOTETomek(sampling_strategy=0.2, tomek=TomekLinks(sampling_strategy=0.4))

In [32]:
%%time
X_train_smote, y_train_smote = smote.fit_resample(X_train_preprocess, y_train)

CPU times: user 16.8 s, sys: 13.7 s, total: 30.5 s
Wall time: 13 s


In [33]:
%%time
X_train_rebalanced, y_train_rebalanced = rus.fit_resample(X_train_smote, y_train_smote)

CPU times: user 1.76 s, sys: 1.22 s, total: 2.97 s
Wall time: 3 s


In [34]:
y_train_rebalanced.value_counts()

0    131903
1     79142
Name: TARGET, dtype: int64

In [35]:
y_train_rebalanced.value_counts()/y_train_rebalanced.shape

0    0.624999
1    0.375001
Name: TARGET, dtype: float64

- Avec une stratégie de SMOTE (sampling à 0.4) et de RUS (sampling à 0.6), Nous passons de 197 857 classe 0 à 131 903 et de 17397 classe 1 à 79 142.
- On s'approche des 60% / 40% pouar la répartition des classes contre 92/8 au début.
- Si j'ai pas voulu avoir une répartition 50/50 c'est par soucis de qualité. Effectivement, les classes sont plus équilibrées mais on a ajouté des clients virtuels qui ressemblaient statistiquement parlant à de vrais clients minoritaires, et on a retiré des clients majoritaires. Equilibré encore plus c'est risqué d'overfit avec ces données là par rapport aux données test sans resampling.

# Modelling and optimization

- Optimisation selon 2 métriques:
    - Score auc 
    - Métrique métier (f beta score)

- Score auc: "L'aire sous la courbe ROC (ou Area Under the Curve, AUC) peut être interprétée comme la probabilité que, parmi deux clients choisis au hasard, un bon et un mauvais, la valeur du marqueur soit plus élevée pour le bon que pour le mauvais."

- Score beta: 
    - F1 score: "Dans l' analyse statistique de la classification binaire , le score F1 ou la mesure F1 est une mesure de la précision d'un test . Il est calculé à partir de la précision et du rappel du test, où la précision est le nombre de vrais résultats positifs divisé par le nombre de tous les résultats positifs, y compris ceux qui ne sont pas identifiés correctement, et le rappel est le nombre de vrais résultats positifs divisé par le nombre de tous les échantillons qui auraient dû être identifiés comme positifs. La précision est également connue sous le nom de valeur prédictive positive et le rappel est également connu sous le nom de sensibilité dans la classification binaire diagnostique."

    - Fbeta score: "Le score F1 est la moyenne harmonique de la précision et du rappel. Le plus générique Fbeta score applique des pondérations supplémentaires, valorisant l'une de la précision ou du rappel plus que l'autre."

#### Dans notre cas, le score auc et la précision du modèle sont importants, mais le plus important reste le recall. Attribué falsieusement la classe 1 à un client, donc la classe "prêt refusé" ou "mauvais client", c'est perdre un bénéfice potentiel. Cependant, attribué falsieusement la classe 0 à un client, donc la classe "prêt accordé" ou "bon client", c'est perdre de l'argent.

#### Nous allons donc prendre tout en compte dans l'optimisation, le score auc, et le score f beta prenant en compte la précision et le recall mais en donnant plus de poids au recall

- comment ?

#### Plus le Beta est petit, plus on valorise la précision, plus le beta est grand, plus on valorise le recall. Nous allons prendre un Beta plus grand que 1 (F1 score), fixé à 2.

In [36]:
## Modelisation
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

## scoring the grid search on the train
# roc_auc par défault
from sklearn.metrics import  make_scorer
from sklearn.metrics import fbeta_score

# Scoring the test (y_true, y_pred and y proba for roc_auc_score)
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
#from sklearn.metrics import fbeta_score

## Baseline model

In [37]:
y_test.value_counts()

0    84825
1     7428
Name: TARGET, dtype: int64

In [38]:
y_train_rebalanced.value_counts()

0    131903
1     79142
Name: TARGET, dtype: int64

- Modèle baseline : DummyClassifier classique avec comme stratégie "uniform"

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

# Baseline model
baseline_model = DummyClassifier(strategy="uniform")
baseline_model.fit(X_train_rebalanced, y_train_rebalanced)
print(f"Score : {baseline_model.score(X_test, y_test)}")

Score : 0.5003848113340488


- Score comme on s'y attendait faible de 0.50

## Random Forest Classifier

In [45]:
def optim_model(model, param_grid, scoring, X, y):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=5)
    grid_search.fit(X, y)
    return grid_search.best_score_, grid_search.best_params_, grid_search.best_estimator_

In [46]:
# Scoring
scorers = {'roc_auc': 'roc_auc', 'fbeta_score': make_scorer(fbeta_score, beta=2)}

In [47]:
# Model
rfc_model = RandomForestClassifier(random_state=0)

# Hyperparameters Grid
n_estimators = [100, 130, 160]
criterions = ['gini', 'entropy']
max_depth = range(2,5)
rfc_parameters = {'n_estimators':n_estimators, 'criterion': criterions, 'max_depth':max_depth}

In [48]:
## Optimisation sur le score auc
# rf_score_auc, rf_param_auc, rf_estimator_auc = optim_model(model=rfc_model, param_grid=rfc_parameters, scoring=scorers['roc_auc'],
#                                                            X=X_train_smtomek, y=y_train_smtomek)

- CPU times: user 2h 6min 55s, sys: 1min 58s, total: 2h 8min 53s
- Wall time: 2h 8min 55s
    
- Score auc: 0.9349012560822322
- Paramètres: 'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 130
- Estimateur optimale au niveau du score auc: RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=130, random_state=0))

In [49]:
## Score f beta
#rf_score_beta, rf_param_beta, rf_estimator_beta = optim_model(model=rfc_model, param_grid=rfc_parameters, scoring=scorers['fbeta_score'],
#                                                  X=X_train_smtomek, y=y_train_smtomek)

- CPU times: user 2h 7min 53s, sys: 1min 56s, total: 2h 9min 49s
- Wall time: 2h 9min 51s
    
- Score f beta: 0.8575190702089286
- Paramètres: 'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 100
- Estimateur optimale au niveau du score f beta: RandomForestClassifier(criterion='entropy', max_depth=4, random_state=0))

## Light Gbm

In [50]:
lgbm_model = lgb.LGBMClassifier(random_state=1)

lgbm_parameters = {'num_leaves':[20, 25, 30], 'max_depth': range(2,5), 'learning_rate': [0.1, 0.3, 0.5], 'n_estimators': [100, 500, 1000]}

In [52]:
import re

#X_train_rebalanced = X_train_rebalanced.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [53]:
## Score auc
# lgbm_score_auc, lgbm_param_auc, lgbm_estimator_auc = optim_model(model=lgbm_model, param_grid=lgbm_parameters, scoring=scorers['roc_auc'],
#                                                                 X=X_train_rebalanced, y=y_train_rebalanced)

- CPU times: user 1d 11h 54min 24s, sys: 8min 47s, total: 1d 12h 3min 11s
- Wall time: 5h 9min 47s

- Score auc: 0.9675222944825889
- Paramètres: 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'num_leaves': 20
- Estimateur optimale au niveau du score auc: LGBMClassifier(max_depth=4, num_leaves=20, random_state=1)

In [54]:
## Score f beta
# lgbm_score_beta, lgbm_param_beta, lgbm_estimator_beta = optim_model(model=lgbm_model, param_grid=lgbm_parameters, scoring=scorers['fbeta_score'], 
#                                                                    X=X_train_rebalanced, y=y_train_rebalanced)

- CPU times: user 1d 12h 47min 9s, sys: 8min 31s, total: 1d 12h 55min 41s
- Wall time: 5h 16min 28s

- Score f beta: 0.9599934906823723,
- Paramètres: 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 1000, 'num_leaves': 30
- Estimateur optimale au niveau du score f beta: LGBMClassifier(max_depth=4, num_leaves=20, random_state=1)

## Comparaison des modèles et performance

- Scores pour le RFC: 
    * Score auc: 0.93
    * Score f beta: 0.86
- Scores pour le LightGBMClassifier: 
    * Score auc: 0.97
    * Score f beta: 0.96
    
#### On garde le modèle LightGBM !

## Valoriser la classe minoritaire 1 grâce aux poids

- 1ère méthode

In [55]:
n_classes = 2
n_samples = y_train.shape[0]
n_samples0 = y_train.value_counts().loc[0]
n_samples1 = y_train.value_counts().loc[1]
w0 = n_samples / (n_classes * n_samples0)
w1 = n_samples / (n_classes * n_samples1)
w0, w1

(0.5439635696487868, 6.186526412599873)

- 2ème méthode

In [56]:
ratio_01 = n_samples0 / n_samples1
ratio_01

11.373052825199746

In [57]:
lgbm_best_model = lgb.LGBMClassifier(learning_rate=0.5, max_depth=3, n_estimators=1000,
                                     num_leaves=30, random_state=2, scale_pos_weight=12)

- Cross validation des données train pour vérifier la stabilité du score et la généralisation du modèle

In [58]:
from sklearn.model_selection import cross_val_score

def eval_model(model, X, y, scoring):
    score = cross_val_score(model, X, y, scoring=scoring,cv=5)
    return score

In [59]:
scoring_metrics = ['roc_auc', make_scorer(fbeta_score, beta=2)]

In [60]:
%%time
score_roc_auc = eval_model(lgbm_best_model, X_train_rebalanced, y_train_rebalanced, scoring_metrics[0])

CPU times: user 28min 56s, sys: 2.27 s, total: 28min 58s
Wall time: 4min 4s


In [61]:
score_roc_auc

array([0.66542871, 0.97361872, 0.99995112, 0.99993141, 0.99974142])

In [62]:
%%time
score_fbeta = eval_model(lgbm_best_model, X_train_rebalanced, y_train_rebalanced, scoring_metrics[1])

CPU times: user 29min, sys: 2.43 s, total: 29min 2s
Wall time: 4min 4s


In [63]:
score_fbeta

array([0.03373724, 0.88462207, 0.90371352, 0.90301786, 0.90227107])

- Scores des données test pour contrôler la cohérence

In [64]:
%%time
lgbm_best_model.fit(X_train_rebalanced, y_train_rebalanced)

CPU times: user 5min 59s, sys: 313 ms, total: 6min
Wall time: 49.5 s


LGBMClassifier(learning_rate=0.5, max_depth=3, n_estimators=1000, num_leaves=30,
               random_state=2, scale_pos_weight=12)

In [65]:
def prediction(model, X, y):
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)
    print('Test accuracy score:', model.score(X, y))
    ras = roc_auc_score(y, y_prob[:,1])
    print('Test roc auc score:', ras)
    ps = precision_score(y, y_pred)
    print('Test precision score:', ps)
    rs = recall_score(y, y_pred)
    print('Test recall score:', rs)
    f1s = f1_score(y, y_pred)
    print('Test f1 score:', f1s)
    f2s = fbeta_score(y, y_pred, beta=2)
    print('Test f2 score:', f2s)
    #auc_score = roc_auc_score(y, y_prob[:,1])
    #print('Test auc score :', auc_score)
    return y_pred,y_prob

In [82]:
y_pred_lgbm, y_proba_lgbm = prediction(lgbm_best_model, X_test_preprocess, y_test)

Test accuracy score: 0.7113047814163225
Test roc auc score: 0.7399228304464782
Test precision score: 0.1656394721264668
Test recall score: 0.6404146472805601
Test f1 score: 0.2632030320635184
Test f2 score: 0.40706132018962543


## Save preprocessor and model

In [67]:
import pickle

filename = 'preprocessor.pkl'
pickle.dump(preprocessor_fitted, open(filename, 'wb'))

filename = 'lgbm_model.pkl'
pickle.dump(lgbm_best_model, open(filename, 'wb'))